In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
p = pd.read_csv('./all_pitches.csv')

In [3]:
ab = pd.read_csv('./all_atbats.csv')

In [4]:
filt = p['zone'] != '-1'
p = p.loc[filt]

In [5]:
p['swing'].dtype

dtype('int64')

In [6]:
#remove take data
filt = p['swing'] == 1
p = p.loc[filt]

In [7]:
p['zone'].value_counts()

4a     178582
7a     152307
3a     141434
15a    134835
5a     134067
8a     120389
12a    119170
6a     113571
13a    102380
1a      92610
0a      70093
2a      60445
10a     26418
16a     22433
15b     17343
14a     13156
13b     10910
12b      9653
16b      8935
10b      8749
14b      5140
9a       1618
11b      1563
9b       1369
Name: zone, dtype: int64

In [8]:
p.head()

,px,py,code,type,pitch_type,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,swing,res,zone
1,-0.191,-0.252,S,S,FF,2015000001,0,1,0,2,0.0,0.0,0.0,1,strike,4a
2,-0.518,0.685,F,S,FF,2015000001,0,2,0,3,0.0,0.0,0.0,1,foul,0a
5,0.627,-0.202,X,X,FF,2015000001,2,2,0,6,0.0,0.0,0.0,1,out,5a
7,-0.257,-0.552,D,X,FC,2015000002,1,0,1,2,0.0,0.0,0.0,1,hit,7a
10,-0.286,-0.769,E,X,FF,2015000003,2,0,1,3,0.0,1.0,0.0,1,hit,6a


In [9]:
filt = ab['stand'] == 'R'
rhh = ab.loc[filt]

In [10]:
#filtering pitches so we have only rhh batters
filt = p['ab_id'].isin(rhh['ab_id'])
p = p.loc[filt]

In [15]:
#setting res of obvious zones to ball
obv_zone = ['9b', '10b', '11b', '12b', '13b', '14b', '15b', '16b']
filt = p['zone'].isin(obv_zone)
p.loc[filt, 'res'] = 'ball'
p['res'].value_counts()

foul      340772
out       225964
strike    215505
hit        93541
ball       38829
Name: res, dtype: int64

In [16]:
swing_trans = {}
for p_name, z in p.groupby('pitch_type'):
    swing_trans[p_name] = {}
    for z_name, g in z.groupby('zone'):
        swing_trans[p_name][z_name] = dict(g['res'].value_counts(normalize=True))

In [17]:
swing_trans['FF']['0a']['strike']

0.25837126137841354

In [18]:
swing_trans

{'CH': {'0a': {'foul': 0.4827272727272727,
   'out': 0.25181818181818183,
   'strike': 0.15636363636363637,
   'hit': 0.10909090909090909},
  '10b': {'ball': 1.0},
  '11b': {'ball': 1.0},
  '12a': {'foul': 0.541802895843064,
   'strike': 0.2524521251751518,
   'out': 0.15506772536198038,
   'hit': 0.05067725361980383},
  '12b': {'ball': 1.0},
  '13a': {'out': 0.3197597385389983,
   'strike': 0.29705856373111916,
   'foul': 0.28928539881635895,
   'hit': 0.09389629891352354},
  '13b': {'ball': 1.0},
  '14a': {'strike': 0.6071428571428571,
   'foul': 0.31887755102040816,
   'out': 0.05229591836734694,
   'hit': 0.021683673469387755},
  '14b': {'ball': 1.0},
  '15a': {'strike': 0.5224321462408739,
   'foul': 0.24895502424343754,
   'out': 0.1717104163183414,
   'hit': 0.05690241319734715},
  '15b': {'ball': 1.0},
  '16a': {'strike': 0.6002615518744551,
   'out': 0.18657367044463818,
   'foul': 0.16434176111595467,
   'hit': 0.04882301656495205},
  '16b': {'ball': 1.0},
  '1a': {'foul': 0.

In [21]:
with open('./swing_transitions.json', 'w') as outfile:
    json.dump(swing_trans, outfile)